# Introduction 

MongoDB est une base de données open source codée en C++ basée sur un concept de stockage sous la forme de documents au format JSON. Le grand avantage de ce système est l'optimisation de la mémoire. Dans une base relationelle, chaque colonne doit être définie au préalable avec une empreinte mémoire et un type de donnée. Dans une base MongoDB si le champ n'est pas présent, il n'apparait pas dans un document et n'impacte pas la mémoire, alors qu'en SQL la place mémoire est utilisée même si le champ est absent, pour spécifier que la valeur est null.

## Les avantages 

- Optimisé pour le multi-machines et la réplication de données ;
- Pas besoin de jointures entre les tables compte tenu du modèle de données sous forme de documents ;
- Un index est créé sur chaque clé pour une rapidité de requêtes ;
- Un langage de requêtage aussi puissant que le SQL ;
- Optimisation de la mémoire .


## Concepts basiques



- Database : une database est un regroupement de collections. Chaque database possède son propre système de fichiers et sa propre authentification. Elle a le même rôle qu'une database en MySQL.
- Collection : une collection est un regroupement de documents. C'est une table en MySQL, la principale différence étant qu'elle ne définit pas un schéma de données fixe. Les documents présents dans une collection n'ont pas forcément tous les mêmes champs.
- Document : un document est un objet JSON stocké sous la forme de plusieurs paires clé:valeur. C'est l'équivalent d'une ligne dans une table SQL.
- Champ : Un champ est l'équivalent d'une colonne en SQL. Il permet de faire des requêtes.

## Identifiants

Tous les documents possèdent un identifiant unique, ce qui permet de le retrouver très efficacement. L'identifiant peut être spécifié lors de l'ajout d'un nouveau document (nom+prenom, adresse email, url, etc). Dans le cas ou aucun identifiant n'est précisé, MongoDB se charge d'en ajouter un. Il est composé d'un nombre stocké sur 12 bytes au format hexadécimal :

- Les 4 premiers bytes sont le timestamp de l'ajout du document
- Les 3 suivants correspondent à l'identifiant de la machine
- Les 2 suivants l'identifiant du processus
- Les 3 derniers sont une valeur incrémentale


## Les types de données

Une base de données MongoDB permet de stocker un grand volume de données hétérogènes sans imposer un modèle de données fixe pour tous les documents. Il est conseillé, comme vu plus haut, de bien définir la structure globale pour garder une cohérence tout au long des développements.

- Integer : entier relatif stocké sur 32 ou 64 bits.
- Double : nombre décimal stocké sur 64 bits.
- String : chaine de caractère (encodée en utf-8)
- Booléen : True ou False
- Object : sous-objet stocké au format JSON
- Date : date au format UNIX (nombre de ms écoulées depuis le 1er janvier 1970) stockée sur 64 bits.
- Array : stocker une liste d'éléments au format atomique ou d'objets

D'autres types sont disponibles et vous pouvez les trouver https://docs.mongodb.com/manual/reference/bson-types/

# Installation


L'installation peut se faire de plusieurs manières.

- Directement depuis les sources ou à partir de packages.  
Liens vers le tutorial https://docs.mongodb.com/manual/tutorial/install-mongodb-on-ubuntu/

Sur une Debian 9 Stretch:

```bash
export http_proxy=http://147.215.1.189:3128
export https_proxy=http://147.215.1.189:3128
apt-get update
apt-get install -y mongodb-org
```

Démarrage du service

Vérifier que le service Mongo est démarré

```bash
> service mongodb status


● mongodb.service - An object/document-oriented database
   Loaded: loaded (/lib/systemd/system/mongodb.service; enabled; vendor preset:
   Active: active (running) since Mon 2018-11-25 13:51:14 CET; 18min ago
     Docs: man:mongod(1)
 Main PID: 22845 (mongod)
    Tasks: 16 (limit: 4915)
   CGroup: /system.slice/mongodb.service
           └─22845 /usr/bin/mongod --unixSocketPrefix=/run/mongodb --config /etc
```

ou le démarrer avec 
 
```bash
> service mongodb start
```


- Ou en instanciant un conteneur Docker. L'avantage de Docker est qu'il n'installe aucune dépendance sur votre machine et laisse son environnement propre. Lien vers le tutorial : https://hub.docker.com/_/mongo/


```bash
> docker run --name my-mongo -d -p 27017:27017 mongo
```

Pour vérifier que le conteneur est bien lancé on peut regarder les logs avec


```bash
> docker logs my-mongo
```

# Connexion


Pour se connecter à une base Mongo deux solutions sont possibles. En ligne de commande ou via un gestionnaire de BDD comme Robo3T https://robomongo.org/ . Dans les deux cas, la syntaxe Mongo est utilisée pour effectuer des requêtes. L'avantage de Robo3T est qu'il possède une interface permettant de visualiser très simplement les données.

Dans un terminal utilisateur standard, la commande mongo permet d'obtenir un shell interactif:

```bash
student@debian:~$ mongo
MongoDB shell version: 3.2.11
connecting to: test
>
```

Le port par défaut de Mongo est le 27017.

# Création d'un modèle de données


La création d'un modèle de données clair et adapté est une tâche importante et primordiale. Ce modèle de données doit être réfléchi à court et long terme et doit prendre en compte la capacité de stockage et les besoins métiers.

## Database


A partir du shell Mongo, on peut afficher les databases disponibles. Au démarrage, aucune n'est créée:
```
> show dbs
admin   0.000GB
config  0.000GB
local   0.000GB
```

Aucun database n'est présente
```
> use test
switched to db test
```

Pour supprimer définitivement une database:

```
> db.dropDatabase()
```
Comme vous pouvez le deviner cette commande est à utiliser avec précaution.

## Collection

Les collections correspondent aux tables en SQL. Elles sont des sous-ensembles d'une database. Pour créer une collection il faut auparavant s'être référencé sur une database.

```
show dbs
use <YOUR_DB_NAME>
db.createCollection(<YOUR_COLLECTION_NAME>)
show collections
```

Comme pour les databases on peut vouloir supprimer définitivement une collection.

```
db.<YOUR_COLLECTION_NAME>.drop()
show collections
```


## Document

Insertion
Un document (objet JSON) est un sous-ensemble d'une collection qui est lui même une sous-partie d'une database. Pour insérer un document il faut donc se référencer sur une database et sur la collection souhaitée.

```
use <YOUR_DB_NAME>
db.createCollection(<YOUR_COLLECTION_NAME>)
show collections
db.<YOUR_COLLECTION_NAME>.insert({
    firstname : "Thomas",
    lastname : "Shelby",
    position : "director",
    company : "Peaky Blinders"})
```

Si vous ne précisez pas d'identifiant unique (id du document), MongoDB se charge de le remplir avec les règles définies précédement. Une bonne pratique est de trouver une règle permettant de retrouver facilement et efficacement un document sans avoir à faire une requête complexe et obliger la base à rechercher dans ses champs. Une technique est de prendre le hash d'une combinaison des champs qui permet de créer une clé unique SHA128(firstname+lastname+position) par exemple.

```
use <YOUR_DB_NAME>
show collections
db.<YOUR_COLLECTION_NAME>.insert({
    firstname : "Thomas",
    lastname : "Shelby",
    position : "CEO",
    gender : "Male",
    age : 35,
    description : "Thomas 'Tommy' Michael Shelby M.P. OBE, is the leader of the Birmingham criminal gang Peaky Blinders and the patriarch of the Shelby Family. His experiences during and after the First World War have left him disillusioned and determined to move his family up in the world.",
    nicknames : ["Tom", "Tommy", "Thomas"],
    company : "Peaky Blinders",
    episodes : [1,2,4,5,6]
    })
```
Pour des soucis de performances, si un grand nombre de documents doivent être insérés très rapidement sans surcharger les appels réseaux, il est possible de passer une liste JSON d'objets à la fonction insert.

```
db.<YOUR_COLLECTION_NAME>.insert([
{
    firstname : "Arthur",
    lastname : "Shelby",
    position : "Associate",
    gender : "Male",
    age : 38,
    description : "Arthur Shelby Jr. is the eldest of the Shelby siblings and the tough member of Peaky Blinders, the Deputy Vice President Shelby Company Limited. He's also a member of the ICA.",
    company : "Peaky Blinders",
    episodes : [1,4,6]

},{
    firstname : "John",
    lastname : "Shelby",
    position : "Associate",
    gender : "Male",
    age : 30,
    description : "John Michael Shelby, also called Johnny or John Boy, was the third of Shelby siblings and a member of the Peaky Blinders.",
    nicknames : ["Johnny", "John Boy"],
    company : "Peaky Blinders",
    episodes : [4,5,6]
},{
    firstname : "Ada",
    lastname : "Thorne",
    position : "HR",
    gender : "Female",
    age : 28,
    description : "Ada Thorne is the fourth and only female of the Shelby sibling. She's the Head of Acquisitions of the Shelby Company Limited.",
    nicknames : ["Ada Shelby"],
    company : "Peaky Blinders",
    episodes : [1,2,6]
},{
    firstname : "Michael",
    lastname : "Gray",
    position : "Accounting",
    gender : "Male",
    age : 21,
    description : "Michael Gray is the son of Polly Shelby, his father is dead, and cousin of the Shelby siblings. He is the Chief Accountant in the Shelby Company Limited.",
    nicknames : ["Henry Johnson", "Jobbie Muncher", "Mickey"],
    company : "Peaky Blinders",
    episodes : [5,6]
},{
    firstname : "Polly",
    lastname : "Gray",
    gender : "Female",
    age : 45,
    position : "CFO",
    description : "Elizabeth Polly Gray (née Shelby) is the matriarch of the Shelby Family, aunt of the Shelby siblings, the treasurer of the Birmingham criminal gang, the Peaky Blinders, a certified accountant and company treasurer of Shelby Company Limited. ",
    nicknames : ["Aunt Polly", "Polly Gray", "Elizabeth Gray", "Polly Shelby", "Pol"],
    company : "Peaky Blinders",
    episodes : [1,2,5,6]
}])
```

# Python 

Il existe une API Python développée pour interagir avec une base de données MongoDB. 

Ce package s'appelle `pymongo`, vous pouvez trouver la documentation https://docs.mongodb.com/getting-started/python/client/. Il est important d'avoir des APIs dans les différents langages pour faciliter l'intégration dans les applications.

Pour installer le package :


In [1]:
!pip install pymongo


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Ce package garde très largement la syntaxe Mongo shell et permet d'utiliser ces méthodes et items (DataBases, Collections, Documents) en tant qu'objets Python.

Dans un premier temps on veut créer le lien entre notre base Mongo et notre programme, pour cela on créé un `client`.

In [16]:
from pymongo import MongoClient
client = MongoClient("localhost",27017)
# client = MongoClient("mongodb://localhost:27017")


Permet de se connecter à une base MongoDB en créant un pointeur client vers cette base. Par défault ce client est paramétré sur le localhost et le port 27017 qui est le port par défaut et très généralement utilisé.
 
Néanmoins, vous pouvez choisir de vous connecter à n'importe quelle base, qu'elle soit sur votre machine ou à distance.


```
client = MongoClient("http://<YOUR_IP_ADDRESS>:<YOUR_PORT_NUMBER>)
```

Il est possible comme depuis le MongoShell de lister les bases de données.


In [22]:
print(client.list_database_names())

['admin', 'config', 'data_engineer_tools', 'local', 'series']



Vous pouvez aussi les sélectionner. Il y a deux syntaxes : 
```
db = client.<YOUR_DATABASE_NAME>
```
ou 
```
db = client["<YOUR_DATABASE_NAME>"]
```


In [19]:
db_series = client.series
print(type(db_series))

<class 'pymongo.database.Database'>


In [23]:
collection_peaky = db_series['peaky'] 

In [24]:
collection_peaky.insert_one(
    {
    "firstname" : "Thomas",
    "lastname" : "Shelby",
    "position" : "CEO",
    "gender" : "Male",
    "age" : 35,
    "description" : "Thomas 'Tommy' Michael Shelby M.P. OBE, is the leader of the Birmingham criminal gang Peaky Blinders and the patriarch of the Shelby Family. His experiences during and after the First World War have left him disillusioned and determined to move his family up in the world.",
    "nicknames" : ["Tom", "Tommy", "Thomas"],
    "company" : "Peaky Blinders",
    "episodes" : [1,2,4,5,6]
    })

InsertOneResult(ObjectId('656f61c920240db3e4b923d9'), acknowledged=True)

In [25]:
DOCUMENTS = [
    {
    "firstname" : "Thomas",
    "lastname" : "Shelby",
    "position" : "CEO",
    "gender" : "Male",
    "age" : 35,
    "description" : "Thomas 'Tommy' Michael Shelby M.P. OBE, is the leader of the Birmingham criminal gang Peaky Blinders and the patriarch of the Shelby Family. His experiences during and after the First World War have left him disillusioned and determined to move his family up in the world.",
    "nicknames" : ["Tom", "Tommy", "Thomas"],
    "company" : "Peaky Blinders",
    "episodes" : [1,2,4,5,6]
    }, 
{
    "firstname" : "Arthur",
    "lastname" : "Shelby",
    "position" : "Associate",
    "gender" : "Male",
    "age" : 38,
    "description" : "Arthur Shelby Jr. is the eldest of the Shelby siblings and the tough member of Peaky Blinders, the Deputy Vice President Shelby Company Limited. He's also a member of the ICA.",
    "company" : "Peaky Blinders",
    "episodes" : [1,4,6]

},{
    "firstname" : "John",
    "lastname" : "Shelby",
    "position" : "Associate",
    "gender" : "Male",
    "age" : 30,
    "description" : "John Michael Shelby, also called Johnny or John Boy, was the third of Shelby siblings and a member of the Peaky Blinders.",
    "nicknames" : ["Johnny", "John Boy"],
    "company" : "Peaky Blinders",
    "episodes" : [4,5,6]
},{
    "firstname" : "Ada",
    "lastname" : "Thorne",
    "position" : "HR",
    "gender" : "Female",
    "age" : 28,
    "description" : "Ada Thorne is the fourth and only female of the Shelby sibling. She's the Head of Acquisitions of the Shelby Company Limited.",
    "nicknames" : ["Ada Shelby"],
    "company" : "Peaky Blinders",
    "episodes" : [1,2,6]
},{
    "firstname" : "Michael",
    "lastname" : "Gray",
    "position" : "Accounting",
    "gender" : "Male",
    "age" : 21,
    "description" : "Michael Gray is the son of Polly Shelby, his father is dead, and cousin of the Shelby siblings. He is the Chief Accountant in the Shelby Company Limited.",
    "nicknames" : ["Henry Johnson", "Jobbie Muncher", "Mickey"],
    "company" : "Peaky Blinders",
    "episodes" : [5,6]
},{
    "firstname" : "Polly",
    "lastname" : "Gray",
    "gender" : "Female",
    "age" : 45,
    "position" : "CFO",
    "description" : "Elizabeth Polly Gray (née Shelby) is the matriarch of the Shelby Family, aunt of the Shelby siblings, the treasurer of the Birmingham criminal gang, the Peaky Blinders, a certified accountant and company treasurer of Shelby Company Limited. ",
    "nicknames" : ["Aunt Polly", "Polly Gray", "Elizabeth Gray", "Polly Shelby", "Pol"],
    "company" : "Peaky Blinders",
    "episodes" : [1,2,5,6]
}]

In [26]:
collection_peaky.insert_many(DOCUMENTS)

InsertManyResult([ObjectId('656f61e020240db3e4b923da'), ObjectId('656f61e020240db3e4b923db'), ObjectId('656f61e020240db3e4b923dc'), ObjectId('656f61e020240db3e4b923dd'), ObjectId('656f61e020240db3e4b923de'), ObjectId('656f61e020240db3e4b923df')], acknowledged=True)

In [28]:
db_series.list_collection_names()

['peaky']

La base de données à bien été créée. Maintenant, pour récupérer un document :

## Requêter

Afin de récupérer les documents stockés dans une collection, des fonctions de requête sont disponibles. La fonction `find()` permet de récupérer les N premiers documents. La fonction `find_one()` permet de récupérer le premier élément.

In [143]:
collection_peaky.find_one()

{'_id': ObjectId('5ddac1ac9ed5d6001efd0430'),
 'firstname': 'Thomas',
 'lastname': 'Shelby',
 'position': 'CEO',
 'gender': 'Male',
 'age': 35,
 'description': "Thomas 'Tommy' Michael Shelby M.P. OBE, is the leader of the Birmingham criminal gang Peaky Blinders and the patriarch of the Shelby Family. His experiences during and after the First World War have left him disillusioned and determined to move his family up in the world.",
 'nicknames': ['Tom', 'Tommy', 'Thomas'],
 'company': 'Peaky Blinders',
 'episodes': [1, 2, 4, 5, 6]}

Cette fonction permet de récupérer le premier élément de la collection sous la forme d'un dictionnaire.


C'est un peu différent pour la méthode find(). Cela crée, pour des raison de performances, un curseur PyMongo. En effet, les données seront récupérées uniquement si elles sont utilisées. C'est intéressant pour des collections très volumineuses.

In [39]:
cursor = collection_peaky.find()
type(cursor)

pymongo.cursor.Cursor

Un curseur est un type d'iterateur python, pour récupérer l'élément suivant:

In [38]:
next(cursor)


StopIteration: 

In [40]:
for document in cursor :
    print('-----')
    print(document)

-----
{'_id': ObjectId('656f611e20240db3e4b923d8'), 'firstname': 'Thomas', 'lastname': 'Shelby', 'position': 'CEO', 'gender': 'Male', 'age': 35, 'description': "Thomas 'Tommy' Michael Shelby M.P. OBE, is the leader of the Birmingham criminal gang Peaky Blinders and the patriarch of the Shelby Family. His experiences during and after the First World War have left him disillusioned and determined to move his family up in the world.", 'nicknames': ['Tom', 'Tommy', 'Thomas'], 'company': 'Peaky Blinders', 'episodes': [1, 2, 4, 5, 6]}
-----
{'_id': ObjectId('656f61c920240db3e4b923d9'), 'firstname': 'Thomas', 'lastname': 'Shelby', 'position': 'CEO', 'gender': 'Male', 'age': 35, 'description': "Thomas 'Tommy' Michael Shelby M.P. OBE, is the leader of the Birmingham criminal gang Peaky Blinders and the patriarch of the Shelby Family. His experiences during and after the First World War have left him disillusioned and determined to move his family up in the world.", 'nicknames': ['Tom', 'Tommy',

Il est possible de passer des arguments à la fonction find() ou find_one().

In [41]:
cur = collection_peaky.find({"lastname":"Shelby"})
next(cur)

{'_id': ObjectId('656f611e20240db3e4b923d8'),
 'firstname': 'Thomas',
 'lastname': 'Shelby',
 'position': 'CEO',
 'gender': 'Male',
 'age': 35,
 'description': "Thomas 'Tommy' Michael Shelby M.P. OBE, is the leader of the Birmingham criminal gang Peaky Blinders and the patriarch of the Shelby Family. His experiences during and after the First World War have left him disillusioned and determined to move his family up in the world.",
 'nicknames': ['Tom', 'Tommy', 'Thomas'],
 'company': 'Peaky Blinders',
 'episodes': [1, 2, 4, 5, 6]}


Les différentes opérations mathématiques sont implémentées. 

- Egalité :  `{key:value}` Correspondance clé valeur entre le champ et la requête. 
- Différence :  `{key: {$ne:value}}`
- Plus (Grand|Petit) que :  les opérateurs sont `$lt` (lower than) ; `$lte` (lower than equals) ; `$gt` (greater than) ; `$gte` (greater than equals) : `{key: {<OPERATEUR>:value}}`.


In [45]:
cur = collection_peaky.find({"age":{"$gte" :30}})
next(cur)

{'_id': ObjectId('656f611e20240db3e4b923d8'),
 'firstname': 'Thomas',
 'lastname': 'Shelby',
 'position': 'CEO',
 'gender': 'Male',
 'age': 35,
 'description': "Thomas 'Tommy' Michael Shelby M.P. OBE, is the leader of the Birmingham criminal gang Peaky Blinders and the patriarch of the Shelby Family. His experiences during and after the First World War have left him disillusioned and determined to move his family up in the world.",
 'nicknames': ['Tom', 'Tommy', 'Thomas'],
 'company': 'Peaky Blinders',
 'episodes': [1, 2, 4, 5, 6]}

Les opérations logiques sont aussi disponibles.

OR `$or` et AND `$and` permettent de faire des requêtes complexes sur une collection. 



In [46]:
cur = collection_peaky.find({"$and":[{"age":{"$gte": 28, "$lt":40}}, {"lastname":"Shelby"}]})
next(cur)

{'_id': ObjectId('656f611e20240db3e4b923d8'),
 'firstname': 'Thomas',
 'lastname': 'Shelby',
 'position': 'CEO',
 'gender': 'Male',
 'age': 35,
 'description': "Thomas 'Tommy' Michael Shelby M.P. OBE, is the leader of the Birmingham criminal gang Peaky Blinders and the patriarch of the Shelby Family. His experiences during and after the First World War have left him disillusioned and determined to move his family up in the world.",
 'nicknames': ['Tom', 'Tommy', 'Thomas'],
 'company': 'Peaky Blinders',
 'episodes': [1, 2, 4, 5, 6]}

### Requêtes complexes

Les objets Mongo peuvent être assez complexes et les requêtes doivent pouvoir matcher tous types de documents.

Pour requêter les valeurs d'une liste : 

In [47]:
cur = collection_peaky.find( { "nicknames":  ["Henry Johnson", "Jobbie Muncher", "Mickey"] } )
next(cur)

{'_id': ObjectId('656f61e020240db3e4b923de'),
 'firstname': 'Michael',
 'lastname': 'Gray',
 'position': 'Accounting',
 'gender': 'Male',
 'age': 21,
 'description': 'Michael Gray is the son of Polly Shelby, his father is dead, and cousin of the Shelby siblings. He is the Chief Accountant in the Shelby Company Limited.',
 'nicknames': ['Henry Johnson', 'Jobbie Muncher', 'Mickey'],
 'company': 'Peaky Blinders',
 'episodes': [5, 6]}

Le champ `nicknames` doit matcher exactement la liste donnée en argument (en contenu et en ordre). 


In [48]:
cur = collection_peaky.find( { "nicknames":  ["Henry Johnson",  "Mickey", "Jobbie Muncher"] } )
next(cur)

StopIteration: 

Lorsqu'aucun document n'a été trouvé le curseur est vide et donc renvoie une erreur lorsque l'on essaye de récupérer le prochain élément.

Si maintenant on veut récupérer tous les documents avec "Mickey" et "Jobbie Muncher", peu importe l'ordre d'apparition et peu importe les autres éléments du tableau.


In [50]:
cur = collection_peaky.find( { "nicknames":  {"$all" :["Mickey", "Jobbie Muncher"] } } )
next(cur)

{'_id': ObjectId('656f61e020240db3e4b923de'),
 'firstname': 'Michael',
 'lastname': 'Gray',
 'position': 'Accounting',
 'gender': 'Male',
 'age': 21,
 'description': 'Michael Gray is the son of Polly Shelby, his father is dead, and cousin of the Shelby siblings. He is the Chief Accountant in the Shelby Company Limited.',
 'nicknames': ['Henry Johnson', 'Jobbie Muncher', 'Mickey'],
 'company': 'Peaky Blinders',
 'episodes': [5, 6]}

On peut vouloir maintenant récupérer tous les documents comptenant "Mickey" dans les nicknames (listes). 


In [52]:
cur = collection_peaky.find( { "nicknames": "Mickey" } )
next(cur)

{'_id': ObjectId('656f61e020240db3e4b923de'),
 'firstname': 'Michael',
 'lastname': 'Gray',
 'position': 'Accounting',
 'gender': 'Male',
 'age': 21,
 'description': 'Michael Gray is the son of Polly Shelby, his father is dead, and cousin of the Shelby siblings. He is the Chief Accountant in the Shelby Company Limited.',
 'nicknames': ['Henry Johnson', 'Jobbie Muncher', 'Mickey'],
 'company': 'Peaky Blinders',
 'episodes': [5, 6]}

Comme on vient de le voir, une requête sur le champ d'une liste se construit de la même manière qu'une requête sur un champ 'basique'.

La syntaxe générique d'une requête Mongo est la suivante.
```
    db.<YOUR_COLLECTION_NAME>.find( { <array field>: { <operator1>: <value1>, ... } })
```

### Limitation, Projection et Tris

Pour des raisons de performances, il peut être intéressant de limiter les accès réseaux. Pour cela, on peut sélectionner les champs devant être retournés (Projection). On peut aussi demander de limiter le nombre de documents (Limitation).

La syntaxe est la suivante : 

```
db.<YOUR_COLLECTION_NAME>.find(QUERY, PROJECTION).LIMIT(N_DOCUMENTS)
```

Un exemple de projection en utilisant les requêtes déjà utilisées plus haut.


In [64]:
cur = collection_peaky.find({"lastname":"Shelby"}, {"position":1})
next(cur)

{'_id': ObjectId('656f611e20240db3e4b923d8'), 'position': 'CEO'}

Avec une requête plus complexe et une autre projection.

In [65]:
cur = collection_peaky.find({"$and":[{"age":{"$gte": 28, "$lt":40}}, {"lastname":"Shelby"}]}, {"firstname":1})
next(cur)

{'_id': ObjectId('656f611e20240db3e4b923d8'), 'firstname': 'Thomas'}

Un exemple de limitation : 

In [66]:
cur = collection_peaky.find({"lastname":"Shelby"}).limit(2)
list(cur)

[{'_id': ObjectId('656f611e20240db3e4b923d8'),
  'firstname': 'Thomas',
  'lastname': 'Shelby',
  'position': 'CEO',
  'gender': 'Male',
  'age': 35,
  'description': "Thomas 'Tommy' Michael Shelby M.P. OBE, is the leader of the Birmingham criminal gang Peaky Blinders and the patriarch of the Shelby Family. His experiences during and after the First World War have left him disillusioned and determined to move his family up in the world.",
  'nicknames': ['Tom', 'Tommy', 'Thomas'],
  'company': 'Peaky Blinders',
  'episodes': [1, 2, 4, 5, 6]},
 {'_id': ObjectId('656f61c920240db3e4b923d9'),
  'firstname': 'Thomas',
  'lastname': 'Shelby',
  'position': 'CEO',
  'gender': 'Male',
  'age': 35,
  'description': "Thomas 'Tommy' Michael Shelby M.P. OBE, is the leader of the Birmingham criminal gang Peaky Blinders and the patriarch of the Shelby Family. His experiences during and after the First World War have left him disillusioned and determined to move his family up in the world.",
  'nickn

Il est aussi possible de passer directement au Nième document avec la fonction `skip()`.

In [67]:
list(collection_peaky.find({"lastname":"Shelby"}))

[{'_id': ObjectId('656f611e20240db3e4b923d8'),
  'firstname': 'Thomas',
  'lastname': 'Shelby',
  'position': 'CEO',
  'gender': 'Male',
  'age': 35,
  'description': "Thomas 'Tommy' Michael Shelby M.P. OBE, is the leader of the Birmingham criminal gang Peaky Blinders and the patriarch of the Shelby Family. His experiences during and after the First World War have left him disillusioned and determined to move his family up in the world.",
  'nicknames': ['Tom', 'Tommy', 'Thomas'],
  'company': 'Peaky Blinders',
  'episodes': [1, 2, 4, 5, 6]},
 {'_id': ObjectId('656f61c920240db3e4b923d9'),
  'firstname': 'Thomas',
  'lastname': 'Shelby',
  'position': 'CEO',
  'gender': 'Male',
  'age': 35,
  'description': "Thomas 'Tommy' Michael Shelby M.P. OBE, is the leader of the Birmingham criminal gang Peaky Blinders and the patriarch of the Shelby Family. His experiences during and after the First World War have left him disillusioned and determined to move his family up in the world.",
  'nickn

In [68]:
cur = collection_peaky.find({"lastname":"Shelby"}).skip(1)
next(cur)

{'_id': ObjectId('656f61c920240db3e4b923d9'),
 'firstname': 'Thomas',
 'lastname': 'Shelby',
 'position': 'CEO',
 'gender': 'Male',
 'age': 35,
 'description': "Thomas 'Tommy' Michael Shelby M.P. OBE, is the leader of the Birmingham criminal gang Peaky Blinders and the patriarch of the Shelby Family. His experiences during and after the First World War have left him disillusioned and determined to move his family up in the world.",
 'nicknames': ['Tom', 'Tommy', 'Thomas'],
 'company': 'Peaky Blinders',
 'episodes': [1, 2, 4, 5, 6]}

On peut trier les résultats récupérés. 

Pour trier dans l'ordre ascendant et donc récupérer le plus jeune de la famille:

In [73]:
cur = collection_peaky.find({"lastname":"Shelby"}, {"firstname":1}).sort([("age", 1)])
next(cur)

{'_id': ObjectId('656f61e020240db3e4b923dc'), 'firstname': 'John'}

Pour trier dans l'ordre descendant :


In [74]:
cur = collection_peaky.find({"lastname":"Shelby"}, {"firstname":1}).sort([("age", -1)])
next(cur)

{'_id': ObjectId('656f61e020240db3e4b923db'), 'firstname': 'Arthur'}

On peut aussi trier selon une clé puis une autre.

In [75]:
cur = collection_peaky.find({"lastname":"Shelby"}, {"firstname":1}).sort([("age", -1), ("firstname", 1)])
next(cur)

{'_id': ObjectId('656f61e020240db3e4b923db'), 'firstname': 'Arthur'}

## Indexation

L'indexation permet d'accélérer les performances sur les requêtes. Si aucun index n'est mis en place, MongoDB doit effectuer un scan de tous les documents pour trouver ceux qui sont pertinents. L'index permet de stocker les valeurs d'un champ de façon triée pour limiter le nombre de document à parcourir pour effectuer une requête. 

Vous pouvez accéder à plus d'informations ici : https://docs.mongodb.com/manual/indexes/

### Indexation simple

L'indexation simple permet de créer l'index en fonction d'un seul champ. 

On spécifie alors l'ordre dans lequel l'index est créé et trié. 

Dans l'ordre croissant, 

In [76]:
collection_peaky.create_index([("age", 1)])

'age_1'

In [77]:
collection_peaky.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]}, 'age_1': {'v': 2, 'key': [('age', 1)]}}

Et dans l'ordre décroissant, 

In [84]:
collection_peaky.create_index([("age", -1)])

'age_-1'

In [79]:
collection_peaky.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'age_1': {'v': 2, 'key': [('age', 1)]},
 'age_-1': {'v': 2, 'key': [('age', -1)]}}

Pour supprimer un index en particulier : 

In [80]:
collection_peaky.drop_index("age_1")

In [81]:
collection_peaky.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'age_-1': {'v': 2, 'key': [('age', -1)]}}

Pour supprimer tous les index : 

In [82]:
collection_peaky.drop_indexes()

In [83]:
collection_peaky.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]}}

Les performances ne sont visibles que pour des collections de taille importante. Mais se ressente très rapidement et sont souvent indispensables. 

### Indexation composée

L'indexation composée permet de créer un index basé sur deux champs différents. L'ordre des champs spécifié dans la création d'un index est important.On peut trier dans l'ordre croissant le premier champ et dans l'ordre décroissant le deuxième champ. 

Il est très utile quand plusieurs champs sont souvent utilisé conjointement pour effectuer des queries. 

In [170]:
collection_peaky.create_index([("age", -1), ("firstname", 1)])

'age_-1_firstname_1'

In [171]:
collection_peaky.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'series.peaky'},
 'age_-1_firstname_1': {'v': 2,
  'key': [('age', -1), ('firstname', 1)],
  'ns': 'series.peaky'}}

In [172]:
Indexations spéciales

SyntaxError: invalid syntax (<ipython-input-172-477be69161df>, line 1)


### Indexations spéciales

Mongo permet plusieurs indexations : 

- Text : permet de faire de la recherche naturelle de *queries* dans du texte. Cet index peut devenir très rapidement très important et prendre beaucoup de place mémoire. Cet index textuel contient un index par mot contenu dans l'ensemble des documents. Il peut aussi être très lent à créer.
- Multiclés : permet de créer un index sur les éléments d'objets stockés dans des listes ou *arrays*.
- 2D, 2DSphere, geoHaystack : permet de créer des index sur des données géospatiales.
- Hash : permet de stocker les valeurs des champs sous forme de *hash*.

Dans ce cours, on se contentera de faire de l'indexation textuelle.

Tous ces mécanismes d'indexation permettent d'accélérer les performances des requêtes. Mais ils peuvent avoir des effets négatifs: 

- Sur l'occupation mémoire : Chaque index doit avoir un minimum de 8 kB et peut prendre beaucoup de place sur le disque et dans la mémoire RAM.
- Sur le temps d'exécution : les opérations d'insertion et d'écriture peuvent être longues puisque Mongo doit insérer chaque nouveau document dans l'index en plus de l'insertion dans la collection.

Exemple : 

Pour créer un index textuel sur la description des personnages : 


In [173]:
collection_peaky.create_index([("description",  "text")])

'description_text'

In [174]:
collection_peaky.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'series.peaky'},
 'age_-1_firstname_1': {'v': 2,
  'key': [('age', -1), ('firstname', 1)],
  'ns': 'series.peaky'},
 'description_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'ns': 'series.peaky',
  'weights': SON([('description', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

Uniquement après que cet index de texte ait été créé, on peut utiliser la méthode `find()` avec l'argument `$text` pour faire une requête dans le texte.


In [175]:
cur = collection_peaky.find( { "$text": { "$search": "female" } } )
next(cur)

{'_id': ObjectId('5ddac4709ed5d6001efd043b'),
 'firstname': 'Ada',
 'lastname': 'Thorne',
 'position': 'HR',
 'gender': 'Female',
 'age': 28,
 'description': "Ada Thorne is the fourth and only female of the Shelby sibling. She's the Head of Acquisitions of the Shelby Company Limited.",
 'nicknames': ['Ada Shelby'],
 'company': 'Peaky Blinders',
 'episodes': [1, 2, 6]}

    
#### Exercice

Supprimez tous les index créés et réessayez de faire la même requête. Que se passe-t-il ?


## Mettre à jour

La mise à jour des documents et une opération très courante dans les bases de données. MongoDB implémente trois fonctions différentes permettant de mettre à jour un ou plusieurs documents à la fois.

- Mettre à jour un seul document : 

```
db.<YOUR_COLLECTION_NAME>.updateOne(<filter>, <update>, <options>)
```

- Le champ `filter` est une requête comme on vient de voir précédemment ; 
- Le champ `update` permet de préciser la requête de mise à jour ;
- Le champ `option` permet de donner des arguments à cette opération.

 Cette fonction va mettre à jour le premier élément renvoyé par la requête `filter`. 
 
 Par exemple : 
 


In [176]:
result = collection_peaky.update_one({"firstname":"Thomas"}, {"$set":{"maincharacter":True}})
result

In [177]:
cur = collection_peaky.find_one({"firstname":"Thomas"})
cur

{'_id': ObjectId('5ddac1ac9ed5d6001efd0430'),
 'firstname': 'Thomas',
 'lastname': 'Shelby',
 'position': 'CEO',
 'gender': 'Male',
 'age': 35,
 'description': "Thomas 'Tommy' Michael Shelby M.P. OBE, is the leader of the Birmingham criminal gang Peaky Blinders and the patriarch of the Shelby Family. His experiences during and after the First World War have left him disillusioned and determined to move his family up in the world.",
 'nicknames': ['Tom', 'Tommy', 'Thomas'],
 'company': 'Peaky Blinders',
 'episodes': [1, 2, 4, 5, 6],
 'maincharacter': True}

Ici le champ update utilise le selecteur `$set` qui permet de définir les couples clé:valeurs à mettre à jour.

- Mettre à jour une liste de documents : 

```
db.<YOUR_COLLECTION_NAME>.update_many(<filter>, <update>, <options>)
```

Cette fonction va mettre à jour tous les documents concernés par la requête `filter`.
Dans l'exemple ci-dessous nous allons mettre à jour tous les éléments correspondant à la family Shelby.


In [178]:
result = collection_peaky.update_many({"lastname":"Shelby"}, {"$set":{"shelbyFamily":True}})
result

In [179]:
cur = collection_peaky.find({"lastname":"Shelby"})
list(cur)

[{'_id': ObjectId('5ddac1ac9ed5d6001efd0430'),
  'firstname': 'Thomas',
  'lastname': 'Shelby',
  'position': 'CEO',
  'gender': 'Male',
  'age': 35,
  'description': "Thomas 'Tommy' Michael Shelby M.P. OBE, is the leader of the Birmingham criminal gang Peaky Blinders and the patriarch of the Shelby Family. His experiences during and after the First World War have left him disillusioned and determined to move his family up in the world.",
  'nicknames': ['Tom', 'Tommy', 'Thomas'],
  'company': 'Peaky Blinders',
  'episodes': [1, 2, 4, 5, 6],
  'maincharacter': True,
  'shelbyFamily': True},
 {'_id': ObjectId('5ddac1ac9ed5d6001efd0431'),
  'firstname': 'Arthur',
  'lastname': 'Shelby',
  'position': 'Associate',
  'gender': 'Male',
  'age': 38,
  'description': "Arthur Shelby Jr. is the eldest of the Shelby siblings and the tough member of Peaky Blinders, the Deputy Vice President Shelby Company Limited. He's also a member of the ICA.",
  'company': 'Peaky Blinders',
  'episodes': [1, 4


L'option `upsert` peut être très intéressante. Elle permet d'ajouter un document si il n'existe pas déjà directement depuis la fonction `update()`. Par défaut, cette option est `false`. 
 
 
```
db.<YOUR_COLLECTION_NAME>.update(<filter>, <update>, {upsert: true})
```
    

## Supprimer 

Pour supprimer des documents, comme pour la mise à jour, il existe deux méthodes : 

- `deleteMany({ <field1>: <value1>, ... }`
- `deleteOne({ <field1>: <value1>, ... }`

Pour supprimer tous les documents possédant une condition : 


In [180]:
result = collection_peaky.delete_many({"lastname": "Gray"})
result

In [181]:
cur = collection_peaky.find({"lastname": "Gray"})
next(cur)

StopIteration: 

Plus aucun document ne possède ces propriétés.

Pour supprimer un seul document (ou le premier si la condition n'est pas assez restrictive) :

In [182]:
result = collection_peaky.delete_one({"firstname": "Arthur"})
result

In [183]:
cur = collection_peaky.find({"firstname": "Arthur"})
next(cur)

{'_id': ObjectId('5ddac4709ed5d6001efd0439'),
 'firstname': 'Arthur',
 'lastname': 'Shelby',
 'position': 'Associate',
 'gender': 'Male',
 'age': 38,
 'description': "Arthur Shelby Jr. is the eldest of the Shelby siblings and the tough member of Peaky Blinders, the Deputy Vice President Shelby Company Limited. He's also a member of the ICA.",
 'company': 'Peaky Blinders',
 'episodes': [1, 4, 6],
 'shelbyFamily': True}

Pour supprimer tous les documents de la collection: 


In [184]:
result = collection_peaky.delete_many({})
result

In [185]:
cur = collection_peaky.find()
next(cur)

StopIteration: 

Plus aucun document n'est présent dans la collection

Quelques choses à savoir : 

- La méthode `deleteMany()` applique une fonction à tous les documents. Toutes les fonctions en Mongo sont atomiques ce qui veut dire qu'elles s'appliquent à chaque document indépendamment les uns des autres.
- La méthode `delete()` ne supprime pas les index, même si on supprime tous les documents de la collection.


### Exercice 

Maintenant que la collection est vide, réintégrez les données précédemment supprimés. 

## Aggregation


Une aggrégation permet de faire des opérations complexes sur des groupes de documents directement dans la base. Elle se charge de grouper les documents entre eux suivant la requête et se charge d'effectuer une opération sur l'ensemble des documents de chacun des groupes. On peut retrouver les mêmes opérations en SQL avec les arguments `GROUP BY`.

La syntaxe est très similaire à toutes les autres fonctions Mongo mais la requête va être plus complexe. 

```
db.<YOUR_COLLECTION_NAME>.aggregate(AGGREGATE_OPERATION)
```

On peut vouloir récupérer le nombre de personnages de chaque famille présente dans la série : 


In [188]:
cur = collection_peaky.aggregate([{"$group" : {"_id" : "$lastname", "charactereNumberByFamily" : {"$sum" : 1}}}])
list(cur)

[{'_id': 'Thorne', 'charactereNumberByFamily': 1},
 {'_id': 'Gray', 'charactereNumberByFamily': 2},
 {'_id': 'Shelby', 'charactereNumberByFamily': 3}]


Vous avez accès à toutes les opérations mathématiques dont vous avez besoin : 

- `$sum` : fait la somme de 
- `$avg` : fait la moyenne 
- `$min` : récupère la valeur minimale 
- `$max` : récupère la valeur maximale 
- `$first` : récupère le premier élément
- `$last` : récupère le dernier élément


In [189]:
cur = collection_peaky.aggregate([{"$group" : {"_id" : "$lastname", "averageAgeByFamily" : {"$avg" : "$age"}}}])
list(cur)


[{'_id': 'Thorne', 'averageAgeByFamily': 28.0},
 {'_id': 'Gray', 'averageAgeByFamily': 33.0},
 {'_id': 'Shelby', 'averageAgeByFamily': 34.333333333333336}]

In [190]:
cur = collection_peaky.aggregate([{"$group" : {"_id" : "$lastname", "minAgeByFamily" : {"$min" : "$age"}}}])
list(cur)


[{'_id': 'Thorne', 'minAgeByFamily': 28},
 {'_id': 'Gray', 'minAgeByFamily': 21},
 {'_id': 'Shelby', 'minAgeByFamily': 30}]

In [191]:
cur = collection_peaky.aggregate([{"$group" : {"_id" : "$lastname", "lastAgeByFamily" : {"$last" : "$age"}}}])
list(cur)


[{'_id': 'Thorne', 'lastAgeByFamily': 28},
 {'_id': 'Gray', 'lastAgeByFamily': 45},
 {'_id': 'Shelby', 'lastAgeByFamily': 30}]

On peut ajouter un paramètre à la fonction `aggregate()` pour filtrer les élements à aggréger.
Si on ne veut récupérer que les hommes : 


In [192]:
cur = collection_peaky.aggregate([
        {"$match":{"gender":"Male"}},
        {"$group" : {"_id" : "$lastname", "averageAgeByFamily" : {"$avg" : "$age"}}}
    ])
list(cur)

[{'_id': 'Gray', 'averageAgeByFamily': 21.0},
 {'_id': 'Shelby', 'averageAgeByFamily': 34.333333333333336}]


Il est aussi possible d'intégrer directement du code JavaScript dans les requêtes Mongo. Des fonctions de Map->Reduce sont disponibles pour effectuer les fonctions d'aggrégation. L'opération de Map->Reduce se découpe en deux phases : 

- Phase de MAP : parcourt tous les élements et extrait les champs voulus.
- Phase de REDUCE : utilise les champs retournés pour effectuer l'opération finale. 

In [193]:
map_function = '''function(){emit(this.lastname, this.age)}'''
reduce_function = '''function(key,values){return Array.sum(values)}'''
out = '''{query :{gender:"Male"}, out:"sumAge"}'''
result = collection_peaky.map_reduce(
        map_function, 
        reduce_function,
        out)

Pour récupérer maintenant les résultats : 

In [194]:
list(result.find())

[{'_id': 'Gray', 'value': 66.0},
 {'_id': 'Shelby', 'value': 103.0},
 {'_id': 'Thorne', 'value': 28.0}]